In [ ]:
# install package
!pip install ions

In [1]:
import io
from ase.io import read

# create string of the .cif file
cif = """
# generated using pymatgen
data_Li3PO4
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   4.89264480
_cell_length_b   6.07079561
_cell_length_c   10.41629663
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   Li3PO4
_chemical_formula_sum   'Li12 P4 O16'
_cell_volume   309.38741088
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_type_symbol
 _atom_type_oxidation_number
  Li+  1.0
  P5+  5.0
  O2-  -2.0
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Li+  Li0  1  0.69534509  0.49846062  0.83568861  1
  Li+  Li1  1  0.19534509  0.50153938  0.66431139  1
  Li+  Li2  1  0.30465491  0.99846062  0.16431139  1
  Li+  Li3  1  0.80465491  0.00153938  0.33568861  1
  Li+  Li4  1  0.30465491  0.50153938  0.16431139  1
  Li+  Li5  1  0.80465491  0.49846062  0.33568861  1
  Li+  Li6  1  0.69534509  0.00153938  0.83568861  1
  Li+  Li7  1  0.19534509  0.99846062  0.66431139  1
  Li+  Li8  1  0.79721728  0.25000000  0.57624687  1
  Li+  Li9  1  0.29721728  0.75000000  0.92375313  1
  Li+  Li10  1  0.20278272  0.75000000  0.42375313  1
  Li+  Li11  1  0.70278272  0.25000000  0.07624687  1
  P5+  P12  1  0.69067243  0.75000000  0.58814555  1
  P5+  P13  1  0.80932757  0.75000000  0.08814555  1
  P5+  P14  1  0.30932757  0.25000000  0.41185445  1
  P5+  P15  1  0.19067243  0.25000000  0.91185445  1
  O2-  O16  1  0.62473411  0.25000000  0.41081835  1
  O2-  O17  1  0.79434655  0.95731182  0.65858046  1
  O2-  O18  1  0.12473411  0.75000000  0.08918165  1
  O2-  O19  1  0.37526589  0.75000000  0.58918165  1
  O2-  O20  1  0.79434655  0.54268818  0.65858046  1
  O2-  O21  1  0.29434655  0.04268818  0.84141954  1
  O2-  O22  1  0.20565345  0.45731182  0.34141954  1
  O2-  O23  1  0.70565345  0.54268818  0.15858046  1
  O2-  O24  1  0.20565345  0.04268818  0.34141954  1
  O2-  O25  1  0.70565345  0.95731182  0.15858046  1
  O2-  O26  1  0.79484118  0.75000000  0.44883378  1
  O2-  O27  1  0.29434655  0.45731182  0.84141954  1
  O2-  O28  1  0.70515882  0.75000000  0.94883378  1
  O2-  O29  1  0.20515882  0.25000000  0.55116622  1
  O2-  O30  1  0.29484118  0.25000000  0.05116622  1
  O2-  O31  1  0.87526589  0.25000000  0.91081835  1
"""

# read structure
atoms = read(io.StringIO(cif), format = 'cif')
atoms

Atoms(symbols='Li12P4O16', pbc=True, cell=[4.8926448, 6.07079561, 10.41629663], spacegroup_kinds=...)

In [ ]:
from ions import Decorator 
from ions.tools import Percolator
from ions.utils import collect_bvse_params

In [20]:
# decorate atoms with oxidation states
# it is highly recommended to use pymatgen's BVAnalyzer in case of large number of structures, the ion's Decorator is too slow.
atoms = Decorator().decorate(atoms) 
atoms = collect_bvse_params(atoms, 'Li', 1) # Li in +1 oxidation state

In [ ]:
specie = 3           # Li
upper_bound = 8.0    # in angstrom

pl = Percolator(atoms, specie, upper_bound)

In [21]:
tr = 0.5             # min allowed edge to framework  distance in Å
cutoff, dim =  pl.mincut_maxdim(tr = tr)
print(f'For {dim}D percolation a cutoff of {cutoff} Å is sufficient')

For 3D percolation a cutoff of 3.03125 Å is sufficient


In [22]:
# unique Li-ion jumps forming 3D percolation network
edges, _ = pl.unique_edges(cutoff, tr)

In [19]:
from ions.tools import SaddleFinder
from ase.optimize import FIRE

for i, edge in enumerate(edges):

    # initial trajectory
    images = edge.superedge(upper_bound).interpolate(n_images = 5)
     
    # create ASE's NEB optimizable
    sf = SaddleFinder()
    neb = sf.bvse_neb(images)

    # define optimizer
    optim = FIRE(neb, logfile=None)

    # optimize trajectory 
    optim.run(fmax=0.1, steps=100)

    # get max force and a migration barrier
    max_force = abs(neb.get_forces()).max()
    print(f'edge_id-{i}', 'fmax:', round(max_force, 3),
           'eV/Angstrom\n', 'Barrier: ', round(sf.get_barrier(images), 3), 'eV\n\n')

edge_id-0 fmax: 0.076 eV/Angstrom
 Barrier:  0.293 eV


edge_id-1 fmax: 0.072 eV/Angstrom
 Barrier:  0.432 eV


edge_id-2 fmax: 0.077 eV/Angstrom
 Barrier:  0.391 eV


edge_id-3 fmax: 0.087 eV/Angstrom
 Barrier:  0.508 eV


